In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nfl_data_py as nfl

In [2]:
base_pbp = nfl.import_pbp_data([2020, 2021, 2022, 2023], downcast=True, cache=False, alt_path=None)

2020 done.
2021 done.
2022 done.
2023 done.
Downcasting floats.


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
pd.set_option('display.max_seq_items', None)

In [5]:
base_pbp.columns

Index(['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team',
       'season_type', 'week', 'posteam', 'posteam_type', 'defteam',
       'side_of_field', 'yardline_100', 'game_date',
       'quarter_seconds_remaining', 'half_seconds_remaining',
       'game_seconds_remaining', 'game_half', 'quarter_end', 'drive', 'sp',
       'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo', 'ydsnet',
       'desc', 'play_type', 'yards_gained', 'shotgun', 'no_huddle',
       'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble', 'pass_length',
       'pass_location', 'air_yards', 'yards_after_catch', 'run_location',
       'run_gap', 'field_goal_result', 'kick_distance', 'extra_point_result',
       'two_point_conv_result', 'home_timeouts_remaining',
       'away_timeouts_remaining', 'timeout', 'timeout_team', 'td_team',
       'td_player_name', 'td_player_id', 'posteam_timeouts_remaining',
       'defteam_timeouts_remaining', 'total_home_score', 'total_away_score',
       'posteam_score'

In [6]:
firsto = base_pbp[base_pbp['game_id'] == '2020_01_ARI_SF']

In [10]:
gamble_check = firsto[['game_seconds_remaining', 'result', 'total', 'spread_line','total_line','home_wp', 'away_wp']]
gamble_check.value_counts()

#use first for all these columns in aggregation

game_seconds_remaining  result  total  spread_line  total_line  home_wp   away_wp 
3600.0                  -4      44     7.0          48.5        0.546262  0.453738    3
303.0                   -4      44     7.0          48.5        0.307735  0.692265    2
1800.0                  -4      44     7.0          48.5        0.628723  0.371277    2
1818.0                  -4      44     7.0          48.5        0.632530  0.367470    2
18.0                    -4      44     7.0          48.5        0.036485  0.963515    2
                                                                                     ..
1211.0                  -4      44     7.0          48.5        0.704887  0.295113    1
1243.0                  -4      44     7.0          48.5        0.694544  0.305456    1
1251.0                  -4      44     7.0          48.5        0.653645  0.346355    1
1255.0                  -4      44     7.0          48.5        0.602853  0.397147    1
1726.0                  -4      44   

In [8]:
gamble_check.sort_values(by = 'game_seconds_remaining', ascending = False)
#when aggregating call FIRST on all categoteries here, obv dont include game seconds

,game_seconds_remaining,result,total,spread_line,total_line,home_wp,away_wp
0,3600.0,-4,44,7.0,48.5,0.546262,0.453738
1,3600.0,-4,44,7.0,48.5,0.546262,0.453738
2,3600.0,-4,44,7.0,48.5,0.546262,0.453738
3,3582.0,-4,44,7.0,48.5,0.579976,0.420024
4,3539.0,-4,44,7.0,48.5,0.604475,0.395525
...,...,...,...,...,...,...,...
185,22.0,-4,44,7.0,48.5,0.038148,0.961852
186,19.0,-4,44,7.0,48.5,0.038148,0.961852
187,18.0,-4,44,7.0,48.5,0.036485,0.963515
188,18.0,-4,44,7.0,48.5,0.036485,0.963515


In [ ]:
#need to find a way to 

In [ ]:
base_use = base_pbp.drop(columns = ['old_game_id', ])